In [3]:
from os.path import join

import numpy as np
import PIL
import rasterio
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

from data_extent import data, folders

In [ ]:
def rasterio_index(rect, src):
    ul = src.index(rect[0], rect[3])
    lr = src.index(rect[1], rect[2])

    minx = min(ul[0], lr[0])
    maxx = max(ul[0], lr[0])
    miny = min(ul[1], lr[1])
    maxy = max(ul[1], lr[1])

    if miny == maxy:
        maxy += 20
        print("I'm bad!", rect[2], rect[3], miny, maxy, minx, maxx)

    return [minx, maxx, miny, maxy]

In [ ]:
def expand_window_no_offset(bbox):
    if bbox[0] - 128 < 0 or bbox[0] + 128 < 0 or bbox[1] - 128 < 0 or \
       bbox[1] + 128 < 0:
        return None
    return [bbox[0] - 128, bbox[0] + 128, bbox[2] - 128, bbox[2] + 128]

In [ ]:
def main():
    # Setting up paths
    datasets_path = '/home/annie/Data/datasets/planet_ships/singapore'
    img_ind = 10  # Select tif #10 for this notebook
    dir_path = join(datasets_path, folders[img_ind]) + '/'
    img_name = data[img_ind][0]
    img_corners = data[img_ind][1]
    src_path = dir_path + 'test_' + str(img_ind) + '.tif'  # warped data
    ogr_path = dir_path + 'ships_ogr_' + str(img_ind) + '.geojson'

    # Opening source tif
    src_ds = rasterio.open(src_path)

    rectangle_js = rectangle['geometry']['coordinates'][0]
    rect = [min(rectangle_js[0][0], rectangle_js[1][0]),  # minX/left
        max(rectangle_js[0][0], rectangle_js[1][0]),  # maxX/right
        min(rectangle_js[0][1], rectangle_js[2][1]),  # minY/bottom
        max(rectangle_js[0][1], rectangle_js[2][1])]   # maxY/top

    bbox_coords =
    with open(ogr_filename) as ogr:
        ogr_dict = json.load(ogr)

    coord_rectangles = [create_coordinate_rectangle(feat) for feat in ogr_dict['features'][::2]]
    pixel_coords = [rasterio_index(rect, src_ds) for rect in coord_rectangles]
    
    ship_boxes = [rasterio.coords.BoundingBox(*bbox_ordered_coords(bbox)) for bbox in bbox_coords]
    chip_coords = [expand_window_with_offset(bbox) for bbox in bbox_coords]
    chip_boxes = [rasterio.coords.BoundingBox(*bbox_ordered_coords(chip)) for chip in chip_coords]
    box_up = [expand_window_no_offset(bbox) for bbox in bbox_coords]
    windows, all_ships = filter_windows(chip_boxes, ship_boxes,
                                        src_ds.shape[0],
                                        src_ds.shape[1])

    masks = [window_ordered_coords(window) for window in windows]


In [ ]:
    with rasterio.open(
            write_path + chip + '.jpg', 'w',
            driver='JPEG', width=256, height=256, count=3,
            dtype='uint8') as dst:
        
        for ship in ships_in_mask:
            write_debug_chip(chip, write_path, data_type, ships_in_mask
                         )
            chip_ships_list.append((chip + '.' + data_type, ship[0],
                               ship[1], ship[2], ship[3], 'ship'))
        
        mask_img = src_ds.read([3, 2, 1], window=mask)
        rescale_img = np.reshape(mask_img, (-1, 1))
        scaler = MinMaxScaler(feature_range=(0, 255))
        rescale_img = scaler.fit_transform(rescale_img)
        mask_img_scaled = (np.reshape(rescale_img,
                           mask_img.shape)).astype(np.uint8)

In [ ]:
        # Create figure and axes
        fig, ax = plt.subplots(1)
        for ship in ships:
            rect = patches.Rectangle((ship[2], ship[0]), ship[3] - ship[2],
                                     ship[1] - ship[0],
                                     linewidth=1, edgecolor='r',
                                     facecolor='none')
            ax.add_patch(rect)

        plt.imshow(mask_img_scaled, origin='upper')

        plt.show()

In [ ]:
main()